# 필요한 패키지 다운받기

In [66]:
#내 환경에 한글폰트 설치(일단 나눔이라는 폰트 1개만)
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent

In [67]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
#matplotlib의 폰트를 Nanum 폰트로 지정합니다

#하고나서 꼭 Ctrl + M을 눌러야함

In [68]:
# 데이터 불러오기(코랩 환경이라 google 마운트를 씁니다)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [69]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/dataton/data/')

In [70]:
!ls 

 진짜최종
'소매 업종 년간 평균 매출 (점심 11~14시)_map.html'
'교육 업종 년간 평균 매출 (점심 11~14시)_map.html'
'음식 업종 년간 평균 매출 (점심 11~14시)_map.html'
'스포츠 업종 년간 평균 매출 (점심 11~14시)_map.html'
'생활서비스업종년간평균매출(점심 11~14시)_map.html'
'소매 업종 년간 평균 매출 (저녁 17~21시)_map.html'
'교육 업종 년간 평균 매출 (저녁 17~21시)_map.html'
'음식 업종 년간 평균 매출 (저녁 17~21시)_map.html'
'스포츠 업종 년간 평균 매출 (저녁 17~21시)_map.html'
'생활서비스업종년간평균매출(점심 17~21시)_map.html'
'소상공인시장진흥공단_상가(상권)정보_서울_202203.csv'
'서울시 우리마을가게 상권분석서비스(상권영역).csv'
'서울시_우리마을가게_상권분석서비스(구_상권배후지_집객시설).csv'
 estimation_sales_data
'교육 업종 년간 평균 매출 주중 주말 비교.html'
 income_consumption_data
 Main컬럼
'소매 업종 년간 평균 매출 주중 주말 비교_map.h

# 필요한 라이브러리 import

In [71]:
import warnings
warnings.filterwarnings('ignore')

import os
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import time
from functools import reduce

!pip install pyproj
import pyproj

import json
import requests
%matplotlib inline 
import seaborn as sns 
sns.set()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [72]:
pwd

'/content/drive/MyDrive/Colab Notebooks/dataton/data'

# 소득소비 데이터 전처리

In [73]:
info = pd.read_csv("./Main컬럼/commercial_info_2021.csv", sep = ',', index_col = 0)
area = pd.read_csv("./Main컬럼/commercial_area2021.csv", sep = ',')
income_17 = pd.read_csv("./income_consumption_data/2017/income_consumption_2017_4.csv", sep = ',')
income_18 = pd.read_csv("./income_consumption_data/2018/income_consumption_2018_4.csv", sep = ',')
income_19 = pd.read_csv("./income_consumption_data/2019/income_consumption_2019_4.csv", sep = ',')
income_20 = pd.read_csv("./income_consumption_data/2020/income_consumption_2020_4.csv", sep = ',')
income_21 = pd.read_csv("./income_consumption_data/2021/income_consumption_2021_4.csv", sep = ',')

In [74]:
com_list = [info, area, income_17, income_18, income_19, income_20, income_21]

In [75]:
for i in com_list:
    print(i.columns)

Index(['상호명', '상권업종대분류코드', '상권업종대분류명', '상권업종중분류코드', '상권업종중분류명', '상권업종소분류코드',
       '상권업종소분류명', '시군구코드', '시군구명', '행정동코드', '행정동명', '법정동코드', '법정동명', '경도',
       '위도'],
      dtype='object')
Index(['기준_년월_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '엑스좌표_값', '와이좌표_값',
       '시군구_코드'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_코드', '월_평균_소득_금액', '지출_총금액', '식료품_지출_총금액',
       '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액',
       '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_코드', '월_평균_소득_금액', '지출_총금액', '식료품_지출_총금액',
       '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액',
       '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_코드', '상권_코드', '월_평균_소득_금액', '지출_총금액', '식료품_지출_총금액',
       '의류_신발_지출_총금액', '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액',
       '문화_지출_총금액', '교육_지출_총금액', '유흥_지출_총금액'],
      dtype='object')
Index(['기준_년_코드', '기준_분기_

In [76]:
income_17 = income_17[income_17.columns[2:]]
income_18 = income_18[income_18.columns[2:]]
income_19 = income_19[income_19.columns[2:]]
income_20 = income_20[income_20.columns[2:]]
income_21 = income_21[income_21.columns[2:]]

In [77]:
income_18

,상권_코드,월_평균_소득_금액,지출_총금액,식료품_지출_총금액,의류_신발_지출_총금액,생활용품_지출_총금액,의료비_지출_총금액,교통_지출_총금액,여가_지출_총금액,문화_지출_총금액,교육_지출_총금액,유흥_지출_총금액
0,2110001,4768273.0,5.204276e+08,1.291260e+08,6.485363e+07,37122367.0,65974602.0,8.834376e+07,22223650.0,14656697.0,7.618534e+07,21941557.0
1,2110002,3117410.0,2.324652e+08,5.499899e+07,2.893209e+07,17084860.0,28271372.0,4.093308e+07,10180461.0,6575012.0,3.580528e+07,9684061.0
2,2110003,3527557.0,1.349699e+09,3.367042e+08,1.661503e+08,98720958.0,170559024.0,2.273671e+08,54707608.0,39088030.0,1.992198e+08,57182240.0
3,2110004,2766807.0,7.127728e+08,2.056497e+08,8.417001e+07,51668351.0,100817261.0,1.118437e+08,23406627.0,22615164.0,8.286516e+07,29736776.0
4,2110005,3642089.0,8.847016e+08,2.214469e+08,1.098824e+08,64738346.0,111988892.0,1.498229e+08,37478670.0,25606895.0,1.265444e+08,37192149.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1585,1001491,3105300.0,3.597924e+09,8.926524e+08,4.529928e+08,300767570.0,434227797.0,6.225060e+08,158308758.0,113915725.0,4.668482e+08,155704784.0
1586,1001492,3005013.0,7.990605e+08,1.985431e+08,1.093387e+08,65537299.0,100540265.0,1.451533e+08,47266880.0,23816404.0,7.339274e+07,35471786.0
1587,1001493,3767553.0,1.889505e+09,4.050555e+08,2.421250e+08,165935630.0,198916229.0,3.278094e+08,100912376.0,55876056.0,3.153858e+08,77489043.0
1588,1001494,2321817.0,1.880810e+09,5.029347e+08,2.421668e+08,151381021.0,253103975.0,3.145640e+08,93975919.0,58322993.0,1.839681e+08,80392486.0


In [78]:
com_list = [ info, area, income_17]
df_2017_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_list[1:])

com_list = [ info, area, income_18]
df_2018_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_list[1:])

com_list = [ info, area, income_19]
df_2019_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_list[1:])

com_list = [ info, area, income_20]
df_2020_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_list[1:])

com_list = [ info, area, income_21]
df_2021_1 = reduce(lambda  left,right: pd.merge(left,right,on=['상권_코드'], how='outer'), com_list[1:])

In [79]:
info = info[['시군구코드', '시군구명']]
info = info.drop_duplicates().reset_index(drop=True)
info.rename(columns={'시군구코드':'시군구_코드'}, inplace=True)

df_2017 = pd.merge(df_2017_1, info, on='시군구_코드')

df_2018 = pd.merge(df_2018_1, info, on='시군구_코드')

df_2019 = pd.merge(df_2019_1, info, on='시군구_코드')

df_2020 = pd.merge(df_2020_1, info, on='시군구_코드')

df_2021 = pd.merge(df_2021_1, info, on='시군구_코드')

In [80]:
def project_array(coord, p1_type, p2_type):
    """
    좌표계 변환 함수
    - coord: x, y 좌표 정보가 담긴 NumPy Array
    - p1_type: 입력 좌표계 정보 ex) epsg:5179
    - p2_type: 출력 좌표계 정보 ex) epsg:4326
    """
    p1 = pyproj.Proj(init=p1_type)
    p2 = pyproj.Proj(init=p2_type)
    fx, fy = pyproj.transform(p1, p2, coord[:, 0], coord[:, 1])
    return np.dstack([fx, fy])[0]

In [81]:
coord_17 = np.array(df_2017[['엑스좌표_값', '와이좌표_값']])
coord_18 = np.array(df_2018[['엑스좌표_값', '와이좌표_값']])
coord_19 = np.array(df_2019[['엑스좌표_값', '와이좌표_값']])
coord_20 = np.array(df_2020[['엑스좌표_값', '와이좌표_값']])
coord_21 = np.array(df_2021[['엑스좌표_값', '와이좌표_값']])

In [82]:
# 좌표계 정보 설정
p1_type = "epsg:5181"
p2_type = "epsg:4326"

# project_array() 함수 실행
result_17 = project_array(coord_17, p1_type, p2_type)
result_18 = project_array(coord_18, p1_type, p2_type)
result_19 = project_array(coord_19, p1_type, p2_type)
result_20 = project_array(coord_20, p1_type, p2_type)
result_21 = project_array(coord_21, p1_type, p2_type)

In [83]:
df_2017['위도_x'] = result_17[:,1]
df_2017['경도_y'] = result_17[:,0]

df_2018['위도_x'] = result_18[:,1]
df_2018['경도_y'] = result_18[:,0]

df_2019['위도_x'] = result_19[:,1]
df_2019['경도_y'] = result_19[:,0]

df_2020['위도_x'] = result_20[:,1]
df_2020['경도_y'] = result_20[:,0]

df_2021['위도_x'] = result_21[:,1]
df_2021['경도_y'] = result_21[:,0]

In [84]:
print(df_2021.columns)

Index(['기준_년월_코드', '상권_구분_코드', '상권_구분_코드_명', '상권_코드', '엑스좌표_값', '와이좌표_값',
       '시군구_코드', '월_평균_소득_금액', '지출_총금액', '식료품_지출_총금액', '의류_신발_지출_총금액',
       '생활용품_지출_총금액', '의료비_지출_총금액', '교통_지출_총금액', '여가_지출_총금액', '문화_지출_총금액',
       '교육_지출_총금액', '유흥_지출_총금액', '시군구명', '위도_x', '경도_y'],
      dtype='object')


In [85]:
df_2017['관광/여가/오락'] = df_2017['여가_지출_총금액'] + df_2017['문화_지출_총금액']
df_2017['생활서비스'] = df_2017['교통_지출_총금액'] + df_2017['의료비_지출_총금액']
df_2017['소매'] = df_2017['생활용품_지출_총금액'] + df_2017['의류_신발_지출_총금액']
df_2017['음식'] = df_2017['식료품_지출_총금액'] + df_2017['유흥_지출_총금액']
df_2017['학문/교육'] = df_2017['교육_지출_총금액']

df_2018['관광/여가/오락'] = df_2018['여가_지출_총금액'] + df_2018['문화_지출_총금액']
df_2018['생활서비스'] = df_2018['교통_지출_총금액'] + df_2018['의료비_지출_총금액']
df_2018['소매'] = df_2018['생활용품_지출_총금액'] + df_2018['의류_신발_지출_총금액']
df_2018['음식'] = df_2018['식료품_지출_총금액'] + df_2018['유흥_지출_총금액']
df_2018['학문/교육'] = df_2018['교육_지출_총금액']

df_2019['관광/여가/오락'] = df_2019['여가_지출_총금액'] + df_2019['문화_지출_총금액']
df_2019['생활서비스'] = df_2019['교통_지출_총금액'] + df_2019['의료비_지출_총금액']
df_2019['소매'] = df_2019['생활용품_지출_총금액'] + df_2019['의류_신발_지출_총금액']
df_2019['음식'] = df_2019['식료품_지출_총금액'] + df_2019['유흥_지출_총금액']
df_2019['학문/교육'] = df_2019['교육_지출_총금액']

df_2020['관광/여가/오락'] = df_2020['여가_지출_총금액'] + df_2020['문화_지출_총금액']
df_2020['생활서비스'] = df_2020['교통_지출_총금액'] + df_2020['의료비_지출_총금액']
df_2020['소매'] = df_2020['생활용품_지출_총금액'] + df_2020['의류_신발_지출_총금액']
df_2020['음식'] = df_2020['식료품_지출_총금액'] + df_2020['유흥_지출_총금액']
df_2020['학문/교육'] = df_2020['교육_지출_총금액']

df_2021['관광/여가/오락'] = df_2021['여가_지출_총금액'] + df_2021['문화_지출_총금액']
df_2021['생활서비스'] = df_2021['교통_지출_총금액'] + df_2021['의료비_지출_총금액']
df_2021['소매'] = df_2021['생활용품_지출_총금액'] + df_2021['의류_신발_지출_총금액']
df_2021['음식'] = df_2021['식료품_지출_총금액'] + df_2021['유흥_지출_총금액']
df_2021['학문/교육'] = df_2021['교육_지출_총금액']

df_2017_ori = df_2017.copy()
df_2018_ori = df_2018.copy()
df_2019_ori = df_2019.copy()
df_2020_ori = df_2020.copy()
df_2021_ori = df_2021.copy()

In [86]:
remain_cols = ['상권_코드', '시군구_코드', '시군구명', '위도_x', '경도_y', '월_평균_소득_금액', '지출_총금액', '관광/여가/오락', '생활서비스',
       '소매', '음식', '학문/교육']

In [87]:
df_2017 = df_2017[remain_cols]
df_2018 = df_2018[remain_cols]
df_2019 = df_2019[remain_cols]
df_2020 = df_2020[remain_cols]
df_2021 = df_2021[remain_cols]

In [88]:
year_data = [2017, 2018, 2019, 2020, 2021]
df = [df_2017, df_2018, df_2019, df_2020, df_2021]

In [89]:
income_avg = list()
comsumption_total = list()
comsumption_orak = list()
comsumption_service = list()
comsumption_retail = list()
comsumption_food = list()
comsumption_education = list()

In [90]:
for i in range(len(df)):

    income_avg.append(df[i][['월_평균_소득_금액', '시군구명']])
    income_avg[i] = income_avg[i].groupby(['시군구명'], as_index=False).sum()
    income_avg[i].sort_values('월_평균_소득_금액', ascending=False, inplace=True)

    comsumption_total.append(df[i][['지출_총금액', '시군구명']])
    comsumption_total[i] = comsumption_total[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_total[i].sort_values('지출_총금액', ascending=False, inplace=True)

    comsumption_orak.append(df[i][['관광/여가/오락', '시군구명']])
    comsumption_orak[i] = comsumption_orak[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_orak[i].sort_values('관광/여가/오락', ascending=False, inplace=True)

    comsumption_service.append(df[i][['생활서비스', '시군구명']])
    comsumption_service[i] = comsumption_service[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_service[i].sort_values('생활서비스', ascending=False, inplace=True)

    comsumption_retail.append(df[i][['소매', '시군구명']])
    comsumption_retail[i] = comsumption_retail[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_retail[i].sort_values('소매', ascending=False, inplace=True)

    comsumption_food.append(df[i][['음식', '시군구명']])
    comsumption_food[i] = comsumption_food[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_food[i].sort_values('음식', ascending=False, inplace=True)

    comsumption_education.append(df[i][['학문/교육', '시군구명']])
    comsumption_education[i] = comsumption_education[i].groupby(['시군구명'], as_index=False).sum()
    comsumption_education[i].sort_values('학문/교육', ascending=False, inplace=True)

    # return comsumption_total, comsumption_orak, comsumption_service, comsumption_retail, comsumption_food, comsumption_education

In [91]:
comsumption_education[1]

,시군구명,학문/교육
17,송파구,2.318941e+10
3,강서구,2.036144e+10
0,강남구,2.018145e+10
21,은평구,1.630984e+10
14,서초구,1.581354e+10
4,관악구,1.572779e+10
1,강동구,1.425273e+10
12,마포구,1.412853e+10
5,광진구,1.280228e+10
18,양천구,1.216560e+10


In [92]:
income_avg[4]

,시군구명,월_평균_소득_금액
0,강남구,375744369.0
14,서초구,283808532.0
19,영등포구,245753067.0
12,마포구,219198398.0
22,종로구,219058258.0
17,송파구,200909835.0
16,성북구,195906777.0
20,용산구,179105199.0
23,중구,177390678.0
10,동대문구,176591507.0


# **Save Data**

In [93]:
pwd

'/content/drive/MyDrive/Colab Notebooks/dataton/data'

In [94]:
# income_avg[4].to_csv('./진짜최종/income_consumption/income_avg.csv',index = False)
# comsumption_total[4].to_csv('./진짜최종/income_consumption/comsumption_total.csv',index = False)
# comsumption_service[4].to_csv('./진짜최종/income_consumption/comsumption_service.csv',index = False)
# comsumption_retail[4].to_csv('./진짜최종/income_consumption/comsumption_retail.csv',index = False)
# comsumption_food[4].to_csv('./진짜최종/income_consumption/comsumption_food.csv',index = False)
# comsumption_education[4].to_csv('./진짜최종/income_consumption/comsumption_education.csv',index = False)

# Folium 지도 시각화

In [95]:
import folium
import requests
import json
from folium.plugins import MarkerCluster
from branca.colormap import linear
from folium import plugins


# 서울 행정구역 json raw파일(githubcontent)
r = requests.get('https://raw.githubusercontent.com/southkorea/seoul-maps/master/kostat/2013/json/seoul_municipalities_geo_simple.json')
c = r.content
seoul_geo = json.loads(c)

### 2021년 서울시 지출 내역

In [96]:
map = folium.Map(location=[37.541, 126.986], zoom_start = 11, tiles='CartoDB positron')

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 지출 총금액',
    data = comsumption_total[-1],
    columns = ['시군구명','지출_총금액'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 지출 총금액",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 [관광/여가/오락] 업종 지출금액',
    data = comsumption_orak[-1],
    columns = ['시군구명','관광/여가/오락'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 관광/여가/오락 지출금액",
    overlay = True,
    Highlight = True
).add_to(map)


folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 [생활서비스] 업종 지출금액',
    data = comsumption_service[-1],
    columns = ['시군구명','생활서비스'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 생활서비스 지출금액",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 [소매] 업종 지출금액',
    data = comsumption_retail[-1],
    columns = ['시군구명','소매'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='properties.name',
    show = False,
    legend_name="지역구별 소매 지출금액",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 [음식] 업종 지출금액',
    data = comsumption_food[-1],
    columns = ['시군구명','음식'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='properties.name',
    #show = False,
    legend_name="지역구별 음식 지출금액",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 [학문/교육] 업종 지출금액',
    data = comsumption_education[-1],
    columns = ['시군구명','학문/교육'],
    fill_color='PuBu', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="지역구별 학문/교육 지출금액",
    overlay = True,
    Highlight = True
).add_to(map)


folium.LayerControl(collapsed = True).add_to(map)

map.save('2021년 서울시 지출 내역_map.html')


## color
#YIGn
#Blues
#BuPu
#PuRd
#YlOrRd
#YlGn

# map.save('map.html')

map

### 년도별 서울시 월 평균 소득금액

In [97]:
map = folium.Map(location=[37.541, 126.986], zoom_start = 11)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2017 서울시 지역별 월 평균 소득금액',
    data = income_avg[0],
    columns = ['시군구명','월_평균_소득_금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2017",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2018 서울시 지역별 월 평균 소득금액',
    data = income_avg[1],
    columns = ['시군구명','월_평균_소득_금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2018",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2019 서울시 지역별 월 평균 소득금액',
    data = income_avg[2],
    columns = ['시군구명','월_평균_소득_금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2019",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2020 서울시 지역별 월 평균 소득금액',
    data = income_avg[3],
    columns = ['시군구명','월_평균_소득_금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2020",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 월 평균 소득금액',
    data = income_avg[4],
    columns = ['시군구명','월_평균_소득_금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2021",
    overlay = True,
    Highlight = True
).add_to(map)

folium.LayerControl(collapsed = True).add_to(map)
map

## color
#YIGn
#Blues
#BuPu
#PuRd
#YlOrRd
#YlGn

### 년도별 서울시 지출 총 금액

In [98]:
map = folium.Map(location=[37.541, 126.986], zoom_start = 11)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2017 서울시 지역별 지출 총금액',
    data = comsumption_total[0],
    columns = ['시군구명','지출_총금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2017",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2018 서울시 지역별 지출 총금액',
    data = comsumption_total[1],
    columns = ['시군구명','지출_총금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2018",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2019 서울시 지역별 지출 총금액',
    data = comsumption_total[2],
    columns = ['시군구명','지출_총금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2019",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2020 서울시 지역별 지출 총금액',
    data = comsumption_total[3],
    columns = ['시군구명','지출_총금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2020",
    overlay = True,
    Highlight = True
).add_to(map)

folium.Choropleth(
    geo_data=seoul_geo,
    name = '2021 서울시 지역별 지출 총금액',
    data = comsumption_total[4],
    columns = ['시군구명','지출_총금액'],
    fill_color='YlOrRd', # 색상 변경도 가능하다
    fill_opacity=0.8,
    line_opacity=0.5,
    key_on='feature.properties.name',
    show = False,
    legend_name="2021",
    overlay = True,
    Highlight = True
).add_to(map)

folium.LayerControl(collapsed = True).add_to(map)
map

## color
#YIGn
#Blues
#BuPu
#PuRd
#YlOrRd
#YlGn

In [99]:
comsumption_total[0]

,시군구명,지출_총금액
17,송파구,1.487313e+11
3,강서구,1.344179e+11
0,강남구,1.262874e+11
4,관악구,1.138744e+11
21,은평구,1.114853e+11
1,강동구,1.041576e+11
14,서초구,1.001392e+11
12,마포구,9.232337e+10
5,광진구,9.199566e+10
24,중랑구,8.545499e+10


In [100]:
comsumption_total[1]

,시군구명,지출_총금액
17,송파구,1.487313e+11
3,강서구,1.344179e+11
0,강남구,1.262874e+11
4,관악구,1.138744e+11
21,은평구,1.114853e+11
1,강동구,1.041576e+11
14,서초구,1.001392e+11
12,마포구,9.232337e+10
5,광진구,9.199566e+10
24,중랑구,8.545499e+10
